In [71]:
import pymongo
from pymongo import MongoClient
import numpy as np
import datetime
from pprint import pprint

In [55]:
URI = "mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000"

def client(uri:str=URI):
    return MongoClient(uri)

def close():
    return client().close()

def get_collection(db:str, collection_name:str):
    clt = client()
    db = clt.get_database(db)
    collection = db.get_collection(collection_name)
    return collection

<h1 style='text-align:center;'> Create a Database and a collection </h1>

In [81]:
def create_dbs(db_name:str):
    try:
        clt = client()
        db = clt[db_name]
        close()
    except Exception as e:
            raise Exception("The following error occurred: ", e)
    
def create_collection(db:str, collection_name:str):
    try:
        clt = client()
        db = clt.get_database(db)
        db.create_collection(collection_name)
        close()
    except Exception as e:
        raise Exception("The following error occurred: ", e)

<h1 style='text-align:center;'> Getting a collection </h1>

In [60]:
try:
    clt = client()
    get_collection(db='to_do_list', collection_name='tasks')
    clt.close()
except Exception as e:
        raise Exception("The following error occurred: ", e)

<h1 style='text-align:center;'> Insert One </h1>

In [61]:
def insert_one(db, collection_name:str, doc:dict):
    try:
        clt = client()
        collection = get_collection(db, collection_name)
        doc = collection.insert_one(doc)
        clt.close()
    except Exception as e:
        raise Exception("The following error occurred: ", e)
    else:
        return doc.inserted_id


In [62]:
doc = {
        "task": "Dormir",
        "description": "Ne rien faire de toute la journée."
    }

print(insert_one(db='to_do_list', collection_name='tasks', doc=doc))

665c5cf11066889440ca3c1b


<h1 style='text-align:center;'> Insert Many </h1>

In [51]:
def insert_many(db:str , collection_name:str, docs:list):
    try:
        clt = client()
        collection = get_collection(db, collection_name)
        for task in docs:
            typ = type(task)
            if typ != dict:
                raise Exception( f'collection has to be a list of dictionaries: {task}')
        data = collection.insert_many(docs)
        client.close()
    except Exception as e:
        raise Exception("The following error occurred: ", e)
    else:
        return data.inserted_ids


In [64]:
# tasks = [
#     {
#         "task": "Acheter du lait",
#         "description": "Aller au supermarché et acheter une bouteille de lait."
#     },
#     {
#         "task": "Appeler le plombier",
#         "description": "Prendre rendez-vous avec le plombier pour réparer une fuite d'eau dans la cuisine."
#     },
#     {
#         "task": "Faire de l'exercice",
#         "description": "Faire une séance de jogging de 30 minutes au parc."
#     },
#     {
#         "task": "Répondre aux e-mails",
#         "description": "Vérifier la boîte de réception et répondre aux e-mails professionnels."
#     },
#     {
#         "task": "Préparer le dîner",
#         "description": "Choisir une recette et préparer le dîner pour ce soir."
#     },
#     {
#         "task": "Lire un livre",
#         "description": "Lire le dernier roman de l'auteur préféré."
#     },
#     {
#         "task": "Faire les courses",
#         "description": "Faire une liste des courses et acheter les produits nécessaires."
#     },
#     {
#         "task": "Nettoyer la maison",
#         "description": "Passer l'aspirateur, faire la poussière et nettoyer les vitres."
#     }
# ]
new_posts = [
    {
        "author": "Mike",
        "text": "Another post!",
        "tags": ["bulk", "insert"],
        "date": datetime.datetime(2009, 11, 12, 11, 14),
    },
    {
        "author": "Eliot",
        "title": "MongoDB is fun",
        "text": "and pretty easy too!",
        "date": datetime.datetime(2009, 11, 10, 10, 45),
    },
]
insert_many(db='to_do_list', collection_name='tasks', docs=new_posts)

[ObjectId('665c5d8a1066889440ca3c1d'), ObjectId('665c5d8a1066889440ca3c1e')]

<h1 style='text-align:center;'> Update One </h1>

In [96]:
def update_one(db, collection_name:str, query_filter:dict, update_operation:dict, upsert=False):
    try:
        clt = client()
        collection = get_collection(db, collection_name)
        if not query_filter and not update_operation:
            raise Exception('query filter and update operation cannot be empty')
        updated_data = collection.update_one(query_filter, update_operation, upsert=upsert)
        clt.close()
    except Exception as e:
        raise Exception("The following error occurred: ", e)
    else:
        if not upsert:
            return updated_data.raw_result
        else:
            return updated_data.upserted_id


<h1 style='text-align:center;'> Update Many </h1>

In [ ]:
def update_many(db, collection_name:str, query_filter:dict, update_operation:dict, upsert=False):
    try:
        clt = client()
        collection = get_collection(db, collection_name)
        if not query_filter and not update_operation:
            raise Exception('query filter and update operation cannot be empty')
        updated_data = collection.update_many(query_filter, update_operation, upsert=upsert)
        clt.close()
    except Exception as e:
        raise Exception("The following error occurred: ", e)
    else:
        if not upsert:
            return updated_data.raw_result
        else:
            return updated_data.upserted_id


In [202]:
update_one(db='to_do_list', collection_name='tasks', query_filter=({'task': 'Dormir et se baigner'}), update_operation={'$set': {'task': 'Dormir et se baigner sans oublier de faire les devoir.'}})

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

<h1 style='text-align:center;'> Delete One and Many </h1>

In [206]:
def delete_one(db, collection_name:str, query_filter:dict):
    try:
        clt = client()
        collection = get_collection(db, collection_name)
        if not query_filter:
            raise Exception('query filter operation cannot be empty')
        deleted_data = collection.delete_one(query_filter)
        clt.close()
    except Exception as e:
        raise Exception("The following error occurred: ", e)
    else:
        return deleted_data.raw_result
    
def delete_many(db, collection_name:str, query_filter:dict):
    try:
        clt = client()
        collection = get_collection(db, collection_name)
        if not query_filter:
            raise Exception('query filter operation cannot be empty')
        deleted_data = collection.delete_many(query_filter)
        clt.close()
    except Exception as e:
        raise Exception("The following error occurred: ", e)
    else:
        return deleted_data.raw_result


In [207]:
delete_one(db='to_do_list', collection_name='tasks', query_filter=({'task': 'Nettoyer la maison'}))

{'n': 1, 'ok': 1.0}

<h1 style='text-align:center;'> Find </h1>

In [199]:
def find_decorator(param=None):
    def decorator(func):
        def wrapper(*args, **kwargs):
            try:
                collection = get_collection(*args, **kwargs)
            except Exception as e:
                raise Exception("The following error occurred: ", e)
            else:
                if param is not None:
                    return collection.find(*param)
                else:
                    return collection.find()
        return wrapper
    return decorator

def list_find_decorator_items(func):
    def items(*args, **kwargs):
        return list(func(*args, **kwargs))
    return items


In [208]:
parameter = ({'task': 'Dormir et se baigner'}, {'task':1, "description":1})
@list_find_decorator_items
@find_decorator()#(param=parameter)
def find(db, collection_name):
    pass

items = find(db='to_do_list', collection_name='tasks')
print(items)

[{'_id': ObjectId('665c5a931066889440ca3c00'), 'task': 'Acheter du lait', 'description': 'Aller au supermarché et acheter une bouteille de lait.'}, {'_id': ObjectId('665c5a931066889440ca3c01'), 'task': 'Appeler le plombier', 'description': "Prendre rendez-vous avec le plombier pour réparer une fuite d'eau dans la cuisine."}, {'_id': ObjectId('665c5a931066889440ca3c02'), 'task': "Faire de l'exercice", 'description': 'Faire une séance de jogging de 30 minutes au parc.'}, {'_id': ObjectId('665c5a931066889440ca3c03'), 'task': 'Répondre aux e-mails', 'description': 'Vérifier la boîte de réception et répondre aux e-mails professionnels.'}, {'_id': ObjectId('665c5a931066889440ca3c04'), 'task': 'Préparer le dîner', 'description': 'Choisir une recette et préparer le dîner pour ce soir.'}, {'_id': ObjectId('665c5a931066889440ca3c05'), 'task': 'Lire un livre', 'description': "Lire le dernier roman de l'auteur préféré."}, {'_id': ObjectId('665c5a931066889440ca3c06'), 'task': 'Faire les courses', '

<h1 style='text-align:center;'> Find and Sort</h1>

In [ ]:
def find_and_sort(data:dict):
    try:
        items = find(db='to_do_list', collection_name='tasks').sort(data)
    except Exception as e:
        raise Exception("The following error occurred: ", e)
    else:
        return list(items)

<h1 style='text-align:center;'> Find and Limit</h1>

In [ ]:
def find_and_limit(limit:int):
    try:
        items = find(db='to_do_list', collection_name='tasks').limit(limit)
    except Exception as e:
        raise Exception("The following error occurred: ", e)
    else:
        return list(items)

<h1 style='text-align:center;'> Find, Sort and Limit</h1>

In [ ]:
def limit_and_sort(limit:int, data:dict):
    try:
        items = find(db='to_do_list', collection_name='tasks').sort(data).limit(limit)
    except Exception as e:
        raise Exception("The following error occurred: ", e)
    else:
        return list(items)

In [86]:
parameter = ({}, {'task': 1, '_id': 0})
list(get_collection(db='to_do_list', collection_name='tasks').find(*parameter))

[{'task': 'Nettoyer la maison'},
 {'task': 'Acheter du lait'},
 {'task': 'Appeler le plombier'},
 {'task': "Faire de l'exercice"},
 {'task': 'Répondre aux e-mails'},
 {'task': 'Préparer le dîner'},
 {'task': 'Lire un livre'},
 {'task': 'Faire les courses'},
 {'task': 'Nettoyer la maison'},
 {'task': 'Acheter du lait'},
 {'task': 'Appeler le plombier'},
 {'task': "Faire de l'exercice"},
 {'task': 'Répondre aux e-mails'},
 {'task': 'Préparer le dîner'},
 {'task': 'Lire un livre'},
 {'task': 'Faire les courses'},
 {'task': 'Nettoyer la maison'},
 {'task': 'Dormir'},
 {},
 {}]

<h1 style='text-align:center;'> Logical Operator</h1>

<p style="margin-bottom:20px; background-color=yellow;">not equal = $ne </p>
<p style="margin-bottom:20px; background-color=yellow;">exit = $exist </p>
<p style="margin-bottom:20px; background-color=yellow;">less than = $lt </p>
<p style="margin-bottom:20px; background-color=yellow;">less than or equal = $lte </p>
<p style="margin-bottom:20px; background-color=yellow;">greater than = $gt </p>
<p style="margin-bottom:20px; background-color=yellow;">greater than or equal = $gte </p>
<p style="margin-bottom:20px; background-color=yellow;">in value = $in </p>
<p style="margin-bottom:20px; background-color=yellow;">not in value = $nin </p>